In [1]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
data = pd.read_csv('cleaned_data.csv', index_col= 0)

In [3]:
data = data.sample(frac = 1)

In [4]:
data.head()

,I/O Data Operations,I/O Data Bytes,Number of subprocesses,Time on processor,Disk Reading/sec,Received Bytes (HTTP),Network packets sent,Network packets received,Pages Input/sec,Page Errors/sec,Confirmed byte radius,Label
ID,,,,,,,,,,,,
ID_MJ5EXDDI1Z,21.833731,5164.821201,27.0,0.208271,0.666340,157.633761,0.844030,0.621917,0.0,773.553754,30.621883,1
ID_BWCY2EXWHR,32.709430,121124.141400,31.0,0.496719,16.816737,40833.970210,16.727877,30.967677,0.0,13576.149690,28.442165,0
ID_SXH210Y4SW,2.267409,831.827915,27.0,0.000000,1.200393,102.255703,0.955869,0.577967,0.0,239.367263,24.999124,0
ID_Q6ZCL1SU4N,5.089690,1130.834289,29.0,0.000000,0.099798,22.454514,0.174646,0.149697,0.0,464.658749,21.287350,1
ID_0JBNAFK5F8,72.685915,15768.381100,28.0,0.763157,1.420861,6165.781413,0.999043,4.773205,0.0,1737.069036,34.166668,1


In [5]:
X = data.drop('Label', axis =1)
y= data['Label']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [7]:
rfc = RandomForestClassifier()

In [8]:
from scipy.stats import randint

params = {'n_estimators': randint(50,500),
          'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': randint(1,20),
            'max_features' : ['sqrt', 'log2', 'None'],
            'bootstrap' : [True, False]
            }


In [9]:
rscv = RandomizedSearchCV(rfc, param_distributions= params, n_iter=5, cv=5)

In [10]:
rscv.fit(X_train, y_train)

/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/sit

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1679e65b0>,
                                        'max_features': ['sqrt', 'log2',
                                                         'None'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1679d2fa0>})

In [11]:
rscv.best_params_

{'bootstrap': False,
 'criterion': 'log_loss',
 'max_depth': 11,
 'max_features': 'sqrt',
 'n_estimators': 442}

In [12]:
first_model = RandomForestClassifier(bootstrap= False, criterion='log_loss', max_depth=11, max_features='sqrt', n_estimators=169)

In [13]:
first_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, criterion='log_loss', max_depth=11,
                       n_estimators=169)

In [14]:
test_output = first_model.predict(X_test)

In [15]:
train_output = first_model.predict(X_train)

In [16]:
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score

In [17]:
score_list = [f1_score , precision_score , recall_score , accuracy_score]
def eval_model(score_list, y_true, y_pred):
    results = []
    for score in score_list:
        value = score(y_true, y_pred)
        results.append({"score_name" : score, "value": value})
    return pd.DataFrame(results)

In [18]:
#performance on test set
eval_model(score_list, y_test, test_output)

,score_name,value
0,<function f1_score at 0x1661f8280>,0.964510
1,<function precision_score at 0x1661f8670>,0.953708
2,<function recall_score at 0x1661f8700>,0.975560
3,<function accuracy_score at 0x1661eeee0>,0.963874


In [19]:
eval_model(score_list, y_train, train_output)

,score_name,value
0,<function f1_score at 0x1661f8280>,0.979199
1,<function precision_score at 0x1661f8670>,0.973656
2,<function recall_score at 0x1661f8700>,0.984805
3,<function accuracy_score at 0x1661eeee0>,0.979135


First Submission

In [20]:
test_data = pd.read_csv('hack4ghana/Test.csv', index_col=0)
test = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [21]:
test

,I/O Data Operations,I/O Data Bytes,Number of subprocesses,Time on processor,Disk Reading/sec,Received Bytes (HTTP),Network packets sent,Network packets received,Pages Input/sec,Page Errors/sec,Confirmed byte radius
ID,,,,,,,,,,,
ID_L7RNFK5JC9,32.709430,1.211241e+05,30.407359,0.496719,3.999064,737.338490,1.844013,1.910664,0.0,726.318848,20.008686
ID_KUTY5K1G5F,2.065990,2.029713e+03,29.000000,0.000000,3.665467,41.119875,0.733093,0.577589,0.0,385.473839,33.015399
ID_H6PJWKU831,114.524684,3.888551e+03,28.000000,0.486075,0.777719,11.332477,0.177764,0.133323,0.0,504.828540,30.091141
ID_DNQXNVTXHE,27.036810,5.581325e+03,25.000000,0.314968,1.789015,13.430214,0.302369,0.125987,0.0,465.950312,22.211252
ID_8CRTF1BJOD,1.177267,1.766345e+02,28.000000,0.000000,1.710369,57.508380,0.888503,0.777440,0.0,771.665166,31.032570
...,...,...,...,...,...,...,...,...,...,...,...
ID_PAWIH292SB,7.290908,3.276107e+03,27.000000,0.034732,0.755765,60.416728,0.955820,0.866907,0.0,332.647660,28.937109
ID_EMPXLFEHRN,12.336515,2.539813e+04,42.000000,1.528201,17.115526,17492.334080,8.535535,17.737909,0.0,15087.869440,28.098248
ID_437MU86JLQ,4.754111,1.123303e+03,30.000000,0.000000,1.932746,28.702390,0.533171,0.377663,0.0,370.087546,31.630495


In [22]:
first_test = test.copy()

In [23]:
first_test["Target"] = first_model.predict(first_test)

In [24]:
submission1 = first_test[["Target"]]

In [25]:
submission1

,Target
ID,
ID_L7RNFK5JC9,0
ID_KUTY5K1G5F,0
ID_H6PJWKU831,1
ID_DNQXNVTXHE,1
ID_8CRTF1BJOD,0
...,...
ID_PAWIH292SB,0
ID_EMPXLFEHRN,0
ID_437MU86JLQ,0


In [26]:
submission1.to_csv('first_submission.csv')

Standardizing Test Data

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [28]:
# new_test['Target'] = first_model.predict(scaled_data)

In [29]:
submission1["Target"].value_counts()

Target
0    2727
1    1091
Name: count, dtype: int64

In [30]:
second_test = test.copy()

In [31]:
index = second_test.index

In [32]:
test_values = second_test.values

In [33]:
columns = second_test.columns

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_values = scaler.fit_transform(test_values)

In [35]:
scaled_test= pd.DataFrame(scaled_values, columns = columns, index = index)

In [36]:
second_test["Target"] = first_model.predict(scaled_test)

In [37]:
submission2 = second_test[['Target']]

In [38]:
submission2['Target'].value_counts()

Target
0    3805
1      13
Name: count, dtype: int64

Standardizing train and test for second model


In [39]:
scaler2 = StandardScaler()
scaler2.fit(X_train)
scaled_x_train = scaler2.transform(X_train)
scaled_x_test = scaler2.transform(X_test)

In [41]:
second_search = rscv.fit(scaled_x_train, y_train)

/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/sit

In [42]:
second_search.best_params_

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 15,
 'max_features': 'sqrt',
 'n_estimators': 81}

In [47]:
second_model = RandomForestClassifier(bootstrap=True, criterion='entropy',max_depth=15, max_features='sqrt',n_estimators=81)

In [48]:
second_model.fit(scaled_x_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=81)

In [49]:
second_test_output = second_model.predict(scaled_x_test)

In [50]:
second_train_output = second_model.predict(scaled_x_train)

In [51]:
#evaluating performance on test data
eval_model(score_list, y_test, second_test_output)

,score_name,value
0,<function f1_score at 0x1661f8280>,0.977330
1,<function precision_score at 0x1661f8670>,0.967099
2,<function recall_score at 0x1661f8700>,0.987780
3,<function accuracy_score at 0x1661eeee0>,0.976941


In [53]:
eval_model(score_list, y_train, second_train_output)

,score_name,value
0,<function f1_score at 0x1661f8280>,0.996705
1,<function precision_score at 0x1661f8670>,0.994302
2,<function recall_score at 0x1661f8700>,0.999119
3,<function accuracy_score at 0x1661eeee0>,0.996705


In [55]:
third_test = second_test.drop('Target', axis = 1)

In [57]:
scaled_test.head()

,I/O Data Operations,I/O Data Bytes,Number of subprocesses,Time on processor,Disk Reading/sec,Received Bytes (HTTP),Network packets sent,Network packets received,Pages Input/sec,Page Errors/sec,Confirmed byte radius
ID,,,,,,,,,,,
ID_L7RNFK5JC9,-0.043571,0.033180,0.100542,0.018357,-0.087224,-0.231242,-0.176934,-0.224539,-0.04958,-0.424566,-2.151766
ID_KUTY5K1G5F,-0.529949,-0.228859,-0.148818,-0.324532,-0.109653,-0.244721,-0.269520,-0.261414,-0.04958,-0.497296,1.143548
ID_H6PJWKU831,1.255016,-0.224769,-0.326001,0.011009,-0.303799,-0.245298,-0.315802,-0.273703,-0.04958,-0.471828,0.402673
ID_DNQXNVTXHE,-0.133608,-0.221044,-0.857551,-0.107107,-0.235808,-0.245257,-0.305417,-0.273906,-0.04958,-0.480124,-1.593735
ID_8CRTF1BJOD,-0.544055,-0.232936,-0.326001,-0.324532,-0.241096,-0.244404,-0.256568,-0.255886,-0.04958,-0.414890,0.641188


In [58]:
third_test = scaled_test.copy()

In [59]:
third_test['Target'] = second_model.predict(third_test)

/Users/josephineamponsah/Documents/hackathon/.venv/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [60]:
third_test['Target'].value_counts()

Target
0    2928
1     890
Name: count, dtype: int64

In [61]:
submission2 = third_test[['Target']]

In [63]:
submission2.to_csv('second_submission.csv')